# Introduction
In this captone project, I will analyze districts in the city of Los Angeles: I will group similar districts in terms of their venue distribution. By doing so, a future entreprenuer should be able to tell most favorable locations in launching his or her business. For example, when a businessman desires to open up a cafe, it would be more beneficial if he does so in the urban cities than rural areas. 

Problem: An entreprenuer is trying to launch a business in LA––but, he or she does not know where to open since each district has differentiated venues. 
# Data
To contrive the data analysis, I would 1)extract data showing coordinates based on different zipcodes of LA from "opendatasoft.com" 2)based on the coordinates, retrieve the necessary data––district, its name, latitude, longitude, and zipcode–– using FourSquare API 3)retrieve the district's neiborghood information(e.g. the number of restaurants and parks) 4) lastly, conduct K-Means clustering using venue frequency as input data. In essence, the purpose of the analysis is to analyze different distributions of venues in districts of LA and suggest some ideas where to open up a new business. 


In [1]:
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim 
import folium